In [ ]:
from selenium import webdriver
from selenium_stealth import stealth
from selenium.webdriver.common.by import By
import time
from tqdm import tqdm
import json

def open_driver():
        options = webdriver.ChromeOptions()
        options.add_argument("start-maximized")
        options.add_argument("--disable-notifications")
        # options.add_argument("--headless")

        options.add_experimental_option("excludeSwitches", ["enable-automation"])
        options.add_experimental_option('useAutomationExtension', False)
        driver = webdriver.Chrome(options=options)

        stealth(driver,
                languages=["en-US", "en"],
                vendor="Google Inc.",
                platform="Win32",
                webgl_vendor="Intel Inc.",
                renderer="Intel Iris OpenGL Engine",
                fix_hairline=True,
                )
        return driver

In [ ]:
with open('results.json', 'r') as f:
    results = json.load(f)

In [ ]:
d = {}

In [ ]:
def foo(driver, ids, d):
    for id in tqdm(ids):
        try:
            time.sleep(1)
            driver.find_element(By.ID, 'search1').send_keys(id)
            driver.find_element(By.ID, 'search1-right').click()
            time.sleep(1)
            driver.find_element(By.CLASS_NAME, 'virtuoso-grid-item').find_element(By.TAG_NAME, 'img').click()
            tmp = {
                'About': {},
                'Attributes': {},
                'Game Info': {}
            }
            time.sleep(1)
            for row in driver.find_element(By.CLASS_NAME, 'about-nft-info').find_elements(By.CLASS_NAME, 'flex'):
                key, value = row.find_elements(By.TAG_NAME, 'div')
                if key.text[:-1] == 'Owner':
                    tmp['About'][key.text[:-1]] = value.find_element(By.TAG_NAME, 'a').get_attribute('href').split('/')[-1]
                elif key.text[:-1] == 'Rank':
                    tmp['About'][key.text[:-1]] = int(value.text)
                elif key.text[:-1] == 'Royalties':
                    tmp['About'][key.text[:-1]] = int(value.text[:-1])
                else:
                    tmp[key.text[:-1]] = value.text
            for row in driver.find_elements(By.CLASS_NAME, 'overflow-auto'):
                key, value = row.find_element(By.TAG_NAME, 'p').text.split(':')
                key, value = key.strip(), value.strip()
                tmp['Attributes'][key] = value
            driver.find_element(By.ID, 'Game Info').click()
            time.sleep(1)
            for row in driver.find_elements(By.CLASS_NAME, 'single-nft-attribute'):
                key, value = row.find_elements(By.TAG_NAME, 'div')
                if key.text[:-1] == 'EarnRate':
                    tmp['Game Info'][key.text[:-1]] = float(value.text)
                else:
                    tmp['Game Info'][key.text[:-1]] = int(value.text)
            driver.find_element(By.CLASS_NAME, 'nft-tab-item').click()
            time.sleep(1)
            driver.find_element(By.ID, 'search1-clear').click()
            driver.execute_script("window.scrollTo(0, 0);")
            driver.find_element(By.CLASS_NAME, 'nft-tab-item').find_element(By.CLASS_NAME, 'absolute').click()
            d[str(id)] = tmp
        except:
            url = f'https://xoxno.com/collection/GSPACEAPE-08bc2b'
            driver.get(url)
    return d

In [ ]:
while d.keys() != results.keys():
    driver = open_driver()
    url = f'https://xoxno.com/collection/GSPACEAPE-08bc2b'
    driver.get(url)
    ids = [x for x in results.keys() if x not in d.keys()]
    d = foo(driver, ids, d)
    driver.close()
    with open('XOXNO.json', 'w') as f:
        json.dump(d, f, indent=4)

In [ ]:
with open('XOXNO.json', 'w') as f:
    json.dump(d, f, indent=4)